## 📦 Setup & Installation

In [1]:
# Check GPU availability
!nvidia-smi

Sat Dec 13 09:16:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Clone repository (if not already cloned)
!git clone https://github.com/mohhajji-1111/combat-racing-rl.git
%cd combat-racing-rl

Cloning into 'combat-racing-rl'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 82 (delta 3), reused 82 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (82/82), 154.09 KiB | 677.00 KiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/combat-racing-rl


In [3]:
# Install dependencies
!pip install -q torch torchvision pygame numpy pyyaml loguru tqdm gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.5 MB/s eta 0:00:00


## 🎮 Train DQN Agent (GPU Accelerated)

In [ ]:
# Train DQN with GPU
!python scripts/train.py --agent dqn --episodes 1000

## 🧠 Train PPO Agent

In [ ]:
# Train PPO
!python scripts/train.py --agent ppo --episodes 1000

2025-12-13 09:20:50 | INFO     | src.utils.logger:setup_logger:83 | Logger initialized with level: INFO
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
2025-12-13 09:20:56 | INFO     | src.utils.helpers:seed_everything:224 | Random seed set to: 42
2025-12-13 09:20:56 | INFO     | src.training.train:main:84 | Starting training with agent: ppo
2025-12-13 09:20:56 | INFO     | src.training.train:main:85 | Configuration: None
2025-12-13 09:20:56 | INFO     | src.training.train:main:86 | Random seed: 42
2025-12-13 09:20:56 | INFO     | src.utils.config_loader:__init__:58 | ConfigLoader initialized with base_dir: /content/combat-racing-rl
2025-12-13 09:20:56 | INFO     | src.utils.config_loader:load:92 | Loading config from: /content/combat-racing-rl/config/game_config.yaml
2025-12-13 09:20:56 | SUCCESS  | src.utils.config_loader:load:110 | Config loaded successfully: game_config.yaml
2025-12-13 09:20:56 | INFO     | src.utils

## 📊 Train Q-Learning Agent

In [ ]:
# Train Q-Learning
!python scripts/train.py --agent qlearning --episodes 1000

## 📈 Check Training Results

In [ ]:
# List saved checkpoints
!ls -lh checkpoints/dqn/
!ls -lh checkpoints/ppo/
!ls -lh checkpoints/qlearning/

## 📹 Record Agent Gameplay (Video)

In [ ]:
# Install video recording dependencies
!apt-get install -y xvfb python3-opengl ffmpeg > /dev/null 2>&1
!pip install -q pyvirtualdisplay

# Setup virtual display for pygame
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1280, 720))
display.start()

print("✅ Virtual display ready!")

In [ ]:
# Create video recording wrapper
import os
os.environ['SDL_VIDEODRIVER'] = 'dummy'

# Play agent and record
!python scripts/play.py --mode agent --agent dqn --checkpoint checkpoints/dqn/final_episode_1000.pt

## 📥 Download Trained Models

In [ ]:
# Download DQN model
from google.colab import files

# Compress checkpoints
!zip -r models.zip checkpoints/

# Download
files.download('models.zip')
print("✅ Models downloaded!")

## 📊 Visualize Training Metrics

In [ ]:
import matplotlib.pyplot as plt
import json
import glob

# Plot training progress
def plot_training_history(agent_type='dqn'):
    # Read training logs
    log_files = glob.glob(f'checkpoints/{agent_type}/training_log*.json')
    
    if not log_files:
        print(f"No logs found for {agent_type}")
        return
    
    episodes = []
    rewards = []
    
    for log_file in log_files:
        with open(log_file, 'r') as f:
            data = json.load(f)
            episodes.extend(data.get('episodes', []))
            rewards.extend(data.get('rewards', []))
    
    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(episodes, rewards, alpha=0.6, label='Episode Reward')
    
    # Moving average
    window = 50
    if len(rewards) >= window:
        moving_avg = [sum(rewards[i:i+window])/window for i in range(len(rewards)-window)]
        plt.plot(range(window, len(rewards)), moving_avg, 'r-', linewidth=2, label=f'{window}-Episode Moving Avg')
    
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.title(f'{agent_type.upper()} Training Progress')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Plot all agents
for agent in ['dqn', 'ppo', 'qlearning']:
    plot_training_history(agent)

## 🎯 Evaluate Agent Performance

In [ ]:
# Evaluate trained agents
!python scripts/evaluate.py --agent dqn --checkpoint checkpoints/dqn/final_episode_1000.pt --episodes 100

## 🏆 Compare Agents

In [ ]:
import pandas as pd

# Create comparison table
results = {
    'Agent': ['Q-Learning', 'DQN', 'PPO'],
    'Episodes': [1000, 1000, 1000],
    'Mean Reward': [0, 457, 0],  # Update with actual values
    'Best Reward': [0, 568, 0],  # Update with actual values
    'Training Time': ['~30min', '6h 42min', '~4h'],
}

df = pd.DataFrame(results)
print("\n" + "="*60)
print("🏆 AGENT COMPARISON RESULTS")
print("="*60)
print(df.to_string(index=False))
print("="*60)

## 💾 Upload to Google Drive (Optional)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy models to Drive
!cp -r checkpoints/ /content/drive/MyDrive/combat_racing_models/
print("✅ Models saved to Google Drive!")

---

## 🎮 Quick Commands Reference

```bash
# Train specific agent
!python scripts/train.py --agent dqn --episodes 500

# Play trained agent
!python scripts/play.py --mode agent --agent dqn --checkpoint checkpoints/dqn/best_model.pt

# Evaluate performance
!python scripts/evaluate.py --agent dqn --episodes 100
```

---

**✨ Built with ❤️ for AI Racing | ENSAM Morocco 🇲🇦**